In [1]:
import os
import pickle
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

from utils_data import Vocabulary, Vectorizer, HeadQA, clean_words, parse_dataset, random_oversamplig, save_dataset_to_pickle, load_dataset_from_pickle
from training import train, validate, evaluate, evaluator, evaluate_better, get_optimizer

from supervised_models import BasicLSTM

%matplotlib inline
%load_ext autoreload
%autoreload 2

C:\Users\tec005m\Anaconda3\envs\afi\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
from datasets import load_dataset

data_es = load_dataset('head_qa', 'es' )

Reusing dataset head_qa (C:\Users\tec005m\.cache\huggingface\datasets\head_qa\es\1.1.0\473dc5357942a3ff52963bd73cad0d167bd1bbc1ca5ca0732ee7372b480dd735)


In [3]:
training, validation, testing = data_es['train'], data_es['validation'], data_es['test']

In [5]:
training_instances = load_dataset_from_pickle('../data/training.pickle')
validation_instances = load_dataset_from_pickle('../data/validation.pickle')
testing_instances = load_dataset_from_pickle('../data/testing.pickle')
mixed_training = load_dataset_from_pickle('../data/mixed_oversampling_training.pickle')

In [6]:
vectorizer = Vectorizer.vectorize_training(mixed_training)
vocab = vectorizer.sentence_vocab
label_vocab = vectorizer.label_vocab

In [7]:
trainset = HeadQA(instances=mixed_training, vectorizer=vectorizer, right_padding=False, max_length=30)
validset = HeadQA(instances=validation_instances, vectorizer=vectorizer, right_padding=False, max_length=30)
testset = HeadQA(instances=testing_instances, vectorizer=vectorizer, right_padding=False, max_length=30)

In [8]:
batch_size = 32
train_dt = DataLoader(trainset, batch_size=batch_size,drop_last=True)
valid_dt = DataLoader(validset, batch_size=batch_size,drop_last=True)
test_dt = DataLoader(testset, batch_size=batch_size,drop_last=True)

In [9]:
torch.random.manual_seed(42)
model = BasicLSTM(len(vocab), 64, trainset.max_length, 1, embedding_dim=100)
optimizer = get_optimizer(model, lr = 0.001, wd = 1e-5)

In [10]:
training_results = train(model, optimizer, train_dt, valid_dt, validate, epochs=50)

C:\Users\tec005m\Anaconda3\envs\afi\lib\site-packages\torch\nn\functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch 0 train loss  0.4147 valid loss 0.023 and accuracy 0.2500
Epoch 1 train loss  0.5952 valid loss 0.017 and accuracy 0.2500
Epoch 2 train loss  0.5801 valid loss 0.016 and accuracy 0.2500
Epoch 3 train loss  0.5876 valid loss 0.012 and accuracy 0.2500
Epoch 4 train loss  0.5821 valid loss 0.010 and accuracy 0.2500
Epoch 5 train loss  0.5932 valid loss 0.011 and accuracy 0.2500
Epoch 6 train loss  0.5985 valid loss 0.009 and accuracy 0.2500
Epoch 7 train loss  0.5818 valid loss 0.011 and accuracy 0.2500
Epoch 8 train loss  0.5492 valid loss 0.011 and accuracy 0.2500
Epoch 9 train loss  0.5690 valid loss 0.011 and accuracy 0.2500
Epoch 10 train loss  0.5532 valid loss 0.014 and accuracy 0.2500
Epoch 11 train loss  0.5580 valid loss 0.014 and accuracy 0.2500
Epoch 12 train loss  0.5166 valid loss 0.020 and accuracy 0.2500
Epoch 13 train loss  0.5309 valid loss 0.014 and accuracy 0.2500
Epoch 14 train loss  0.5048 valid loss 0.013 and accuracy 0.2500
Epoch 15 train loss  0.4910 valid l

In [11]:
acc, points = evaluate(model, validation, trainset.encode, evaluator)
acc, points

(tensor([0.2621]), 66)

In [12]:
acc, points = evaluate(model, testing, trainset.encode, evaluator)
acc, points

(tensor([0.2615]), 126)

In [13]:
save_dataset_to_pickle('results_v2/train_results_lstm.pickle', training_results)
training_results = load_dataset_from_pickle('results_v2/train_results_lstm.pickle')

In [14]:
model_path = os.getcwd() + '/trained_models_v2/lstm'
torch.save(model.state_dict(), model_path)

In [15]:
model = BasicLSTM(len(vocab), 64, trainset.max_length, 1, embedding_dim=100)
model.load_state_dict(torch.load(model_path))
model.eval()

BasicLSTM(
  (embeddings): Embedding(28831, 100, padding_idx=0)
  (lstm): LSTM(100, 64, batch_first=True)
  (linear): Linear(in_features=64, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [16]:
acc, points, acc_list, points_list = evaluate_better(model, validation, trainset.encode, evaluator)
acc, points, acc_list, points_list

(0.26219568,
 11.0,
 [tensor(0.2965),
  tensor(0.2522),
  tensor(0.2622),
  tensor(0.2468),
  tensor(0.2743),
  tensor(0.2412)],
 [42, 2, 11, -3, 22, -8])

In [17]:
acc, points, acc_list, points_list = evaluate_better(model, testing, trainset.encode, evaluator)
acc, points, acc_list, points_list

(0.26162377,
 10.5,
 [tensor(0.2588),
  tensor(0.2691),
  tensor(0.2675),
  tensor(0.2328),
  tensor(0.2652),
  tensor(0.2597),
  tensor(0.2788),
  tensor(0.2500),
  tensor(0.2926),
  tensor(0.2208),
  tensor(0.2756),
  tensor(0.2687)],
 [8, 17, 16, -16, 14, 9, 26, 0, 39, -27, 23, 17])